In [1]:
# Python Library and modules
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, 
                             precision_score, recall_score)
import torch
import torch.nn as nn
import torch.optim as optim

# Self Modules
from model import AlexNet, evaluate
from preprocess import preprocess

### Create the model

In [2]:
model = AlexNet(5)

### Preprocess the Data

In [3]:
train_dataloader, val_dataloader ,test_dataloader = preprocess()

### Training Loop

In [4]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)

train_acc = []
val_acc = []
# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    train_total = 0
    train_correct = 0
    for images, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += labels.eq(predicted).sum().item()
    
    running_accuracy = 100* train_correct/train_total
    train_acc.append(running_accuracy)
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}] - "
          f"Validation Loss: {val_loss/len(val_dataloader):.4f}")
    print("Train Accuracy",train_acc[-1])
    val_acc.append(100*correct/total)

Epoch [1/25] - Validation Loss: 1.5134
Train Accuracy 34.05
Epoch [2/25] - Validation Loss: 1.2145
Train Accuracy 45.45
Epoch [3/25] - Validation Loss: 1.3361
Train Accuracy 51.4
Epoch [4/25] - Validation Loss: 1.1685
Train Accuracy 53.55
Epoch [5/25] - Validation Loss: 0.9798
Train Accuracy 58.05
Epoch [6/25] - Validation Loss: 1.0039
Train Accuracy 63.65
Epoch [7/25] - Validation Loss: 0.8461
Train Accuracy 70.25
Epoch [8/25] - Validation Loss: 0.9620
Train Accuracy 68.3
Epoch [9/25] - Validation Loss: 0.7732
Train Accuracy 71.35
Epoch [10/25] - Validation Loss: 0.9557
Train Accuracy 76.25
Epoch [11/25] - Validation Loss: 0.7542
Train Accuracy 77.05
Epoch [12/25] - Validation Loss: 0.7559
Train Accuracy 77.05


KeyboardInterrupt: 

### Learning Visualizations

#### Training Accuracy

In [ ]:
epochs=[i+1 for i in range(25)]

plt.plot(epochs, train_acc)

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('The Training Accuracy')

# Show the plot
plt.show()

#### Validation Accuracy

In [ ]:
epochs=[i+1 for i in range(25)]

plt.plot(epochs, val_acc)

plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('The Validation Accuracy')

# Show the plot
plt.show()

### Save the model

In [ ]:
torch.save(model,'./model/trained_model.pth')

### Load the model

In [ ]:
# Load the saved model
loaded_model = torch.load('./model/trained_model.pth')
loaded_model.eval()  # Set the model to evaluation mode if needed

#### Model Evaluation

##### The original model

In [ ]:
trues, preds = evaluate(test_dataloader, model, criterion)

In [ ]:
print("The accuracy is: ", accuracy_score(trues, preds))

In [ ]:
print("The precision is: ", precision_score(trues, preds))

In [ ]:
print("The recall score is: ", recall_score(trues, preds))

In [ ]:
print("The F1 score is ", f1_score(trues, preds, average=None))

In [ ]:
sns.heatmap(confusion_matrix(trues, preds), annot=True)

#### The loaded model